In the previous part, we were **manually** executing the tasks below on a **single CSV file**:
- Import a CSV file into a Pandas df
- Clean the table name and remove extra symbols, spaces, capital letters
- Clean the column names and remove extra symbols, spaces, capital letters
- Write the create table SQL statement
- Connect to the database
- Drop tables with same name (avoid conflicts)
- Create table
- Save dafaframe to CSV
- Open the CSV file and save it as an object
- Upload to database

Our goal in this notebook is to **automate everything (using for-loops) and handle multiple CSV files**.

In [1]:
import os
import numpy as sp
import pandas as pd
import mysql.connector as mysql

### Find only CSV files in current directory 

In [2]:
csv_files = []
for file in os.listdir():
    if file.endswith('.csv'):
        csv_files.append(file)
csv_files

['Customer Contracts$.csv', 'Customer Demo.csv', 'Customer Engagements.csv']

### Create a new directory

In [3]:
dataset_dir = 'datasets'
try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

### Move them to a new directory

In [4]:
import shutil
source = "C:/Users/User/OneDrive - Nanyang Technological University/CSV to DB automation/"
destination = "C:/Users/User/OneDrive - Nanyang Technological University/CSV to DB automation/datasets/"

for file in csv_files:
    shutil.move(source + file, destination + file)

### Create Pandas dataframes from the CSV files

In [5]:
data_path = os.getcwd() + "/" + dataset_dir + "/"
df = {}
print("Dataframes have been created for the following file(s):")
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path + file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path + file, encoding = "ISO-8859-1")
    print(file)

Dataframes have been created for the following file(s):
Customer Contracts$.csv
Customer Demo.csv
Customer Engagements.csv


### Clean the name of each tables and column headers (remove extra symbols, spaces, capital letters)

In [6]:
replacements = {
    'object': 'VARCHAR(255)',
    'float64': 'FLOAT',
    'int64': 'INT',""
    'datetime64': 'DATETIME',
    'timedelta64[ns]': 'VARCHAR(255)'
}

In [7]:
# create a database manually before executing the query below
db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "root",
    database = 'csvtodbauto'
)
cursor = db.cursor()
print('Opened database successfully')

Opened database successfully


In [8]:
for k in csv_files:
    dataframe = df[k]
    clean_tbl_name = k.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
    # Remove .csv extension from the cleaned table name
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    print(tbl_name)
    
    # Rename column headers
    new_name = k.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
    dataframe.rename(columns = {'i':'new_name'}, inplace = True)
    
    # Table schema data types
    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip (dataframe.columns, dataframe.dtypes.replace(replacements)))
    
    # Connect to database
    db = mysql.connect(
        host = "127.0.0.1",
        user = "root",
        passwd = "root",
        database = 'csvtodbauto'
    )
    cursor = db.cursor()
    print('Opened database successfully')
    
    cursor.execute("DROP TABLE IF EXISTS %s" % (clean_tbl_name))
    
    # Create table
    cursor.execute("CREATE TABLE IF NOT EXISTS %s (%s);" % (tbl_name, col_str))
    print('Table {0} was created successfully.'.format(tbl_name))
    df[k] = df[k].apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('N/A'))
    
    for i, row in df[k].iterrows():
        param = ""
        for i in range (len(df[k].columns)):
            param = param + "%s, "

        param = param[:-2]
        sql = "INSERT INTO csvtodbauto.%s VALUES (%s)" % (tbl_name, param)
        cursor.execute(sql, tuple(row))
        
    db.commit()
    cursor.close()
    print('Table {0} imported to database'.format(tbl_name) + '\n')
    
print('All tables have been successfully imported into the database')

customer_contracts
Opened database successfully
Table customer_contracts was created successfully.
Table customer_contracts imported to database

customer_demo
Opened database successfully
Table customer_demo was created successfully.
Table customer_demo imported to database

customer_engagements
Opened database successfully
Table customer_engagements was created successfully.
Table customer_engagements imported to database

All tables have been successfully imported into the database
